<a href="https://colab.research.google.com/github/tylerasteele/LinearAlgebraProjects/blob/main/Copy_of_Sports_Teams_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the preliminary code

In [1]:
import matplotlib.pyplot as plt
from sympy import Poly
import sympy as sym
import numpy as np
from numpy import c_

# We'll specify that x and t are variables:
x, t = sym.symbols('x, t')

ll = "loaded"

In [22]:
lll = " correctly!"

def rref(matrix):
    A = np.array(matrix, dtype=np.float64)
    
    r = 0 #row
    c = 0 #column

    (maxr, maxc) = A.shape

    prev_row_pivot = -1   # The previous row with a leading term:
    tmp_pivot_row = -1
    this_pivot_row = -1

    for c in range(maxc):  # go through each column
      tmp_pivot_row = -1  # reset the tmp pivot row
      this_pivot_row = -1  # reset this pivot row
      for r in range(maxr):  # Find the leading term in this column below prev_row_pivot
        if ((r > prev_row_pivot) and (A[r][c] != 0.0)):
          tmp_pivot_row = r  # find the new pivot row
      if (tmp_pivot_row > -1):  # we found a candidate for the leading term:
        #swap row this_pivot_row with (prev_row_pivot + 1)
        A[[tmp_pivot_row, prev_row_pivot+1]] = A[[prev_row_pivot + 1, tmp_pivot_row]]
        #set this_pivot_row to prev_row_pivot+1
        this_pivot_row = prev_row_pivot + 1
        #Now, normalize the row based on the pivot value:
        A[ this_pivot_row ] = A[ this_pivot_row ] / A[ this_pivot_row ][c]
        #Then, use this row to force 0's above and below the pivot entry:
        #subtract multiples of of the this_pivot_row row from the others
        for r_ in range(len(A)):
            if r_ != this_pivot_row:
                A[r_] = A[r_] - A[this_pivot_row] * A[r_][c] / A[this_pivot_row][c]
        prev_row_pivot = this_pivot_row
    return A   


def is_singular(matrix):
  singular = True
  notsingular = False

  (mr, mc) = matrix.shape
  
  return_value = notsingular
  if (mc - np.linalg.linalg.matrix_rank(matrix) > 0) :
    return_value = singular
  return return_value
 
    
    

print(">> The page ", ll, lll)

>> The page  loaded  correctly!


# Ranking Teams based on games

From our packet, we saw there were five games:

Home  | Away
------|------
Winona State 16 | Sioux Falls 15
Winona State 34 | Augustana 20
Sioux Falls 51  | Augustana 13
Duluth 28       |  Augustana 17
Winona State 17 | Duluth 37

Let's encode this system into a matrix.

The unknown rankings are $r_w, r_s, r_a, r_d$.

The system looks like $M \vec{x} = \vec{b}$, or

$$
\left( \begin{array}{cccc} 1 & -1 & 0 & 0 \\ 1 & 0 & -1 & 0 \\ 0 & 1 & -1 & 0 \\ 0 & 0 & -1 & 1 \\ -1 & 0  & 0 & 1 \\ \end{array} \right) \, \left( \begin{array}{c} r_w \\ r_s \\ r_a \\ r_d \\ \end{array} \right) = \left( \begin{array}{c} 1 \\ 14 \\ 38 \\ 11 \\ 20 \\ \end{array} \right)$$

We'll solve this system using least squares 
$$M^T I M \vec{x} = M^T I \vec{b} $$

$$ \vec{x} = \left( M^T I M  \right)^{-1} M^T I \vec{b} $$






In [23]:
# create the matrix:
M = np.matrix([[1, -1, 0, 0],
               [1, 0, -1, 0],
               [0, 1, -1, 0],
               [0, 0, -1, 1],
               [-1, 0, 0, 1]])


# create the vector for the right hand side:
pts = np.matrix([[1], [14], [38], [11], [20]])
 
#Make sure the system looks appropriate:
print(np.c_[M,pts])

[[ 1 -1  0  0  1]
 [ 1  0 -1  0 14]
 [ 0  1 -1  0 38]
 [ 0  0 -1  1 11]
 [-1  0  0  1 20]]


In [24]:
# Generate the transpose of M:
MT = M.transpose()

print("The Transpose of M is:")
print(MT)

# and find MT*M and MT*pts:
print("\n")
print("MT * M is: ")
MTM = MT * M
print(MTM)

print("\n")
print("MT * pts is:")
MTpts = MT * pts
print(MTpts)


The Transpose of M is:
[[ 1  1  0  0 -1]
 [-1  0  1  0  0]
 [ 0 -1 -1 -1  0]
 [ 0  0  0  1  1]]


MT * M is: 
[[ 3 -1 -1 -1]
 [-1  2 -1  0]
 [-1 -1  3 -1]
 [-1  0 -1  2]]


MT * pts is:
[[ -5]
 [ 37]
 [-63]
 [ 31]]


If $M^T M$ has an inverse, then we can use it.  But, if $M^T M$ is singular, then it doesn't have an inverse.  In this situation, we need to replace every entry in one row (say the bottom row) of $M^T M$ and $M^T \vec{pts}$ with 1's.  (This knocks the system out of *not* having an inverse, so it will now have an inverse.)

In [25]:
#  Check if MTM is singular
is_singular(MT*M)

True

In [26]:
# If MTM is singular, then adjust one row of MTM and MT pts to be just 1's:

#MTM[3] is the entire 4th row of MTM, 
#so MTM[3] = 1 will put 1's in MTM's 4th row.

MTM[3] = 1
MTpts[3] = 1

print("MTM is now: ")
print(MTM)

print("and MTpts is now:")
print(MTpts)

print("Is our modified MTM singular?")
is_singular(MTM)

MTM is now: 
[[ 3 -1 -1 -1]
 [-1  2 -1  0]
 [-1 -1  3 -1]
 [ 1  1  1  1]]
and MTpts is now:
[[ -5]
 [ 37]
 [-63]
 [  1]]
Is our modified MTM singular?


False

In [27]:
#Find our solution using the inverse of MTM:

#First, find the inverse of MTM:
MTMI = np.linalg.inv(MTM)

print("The inverse of MTM is:")
print(MTMI)

#Now, use the inverse of MTM to solve for the rankings:

print("\n")
print("The solution are rankings:")
MTMI * MTpts

The inverse of MTM is:
[[ 2.50000000e-01  1.85037171e-17  1.85037171e-17  2.50000000e-01]
 [ 1.25000000e-01  5.00000000e-01  1.25000000e-01  2.50000000e-01]
 [-1.38777878e-17 -2.77555756e-17  2.50000000e-01  2.50000000e-01]
 [-3.75000000e-01 -5.00000000e-01 -3.75000000e-01  2.50000000e-01]]


The solution are rankings:


matrix([[ -1.  ],
        [ 10.25],
        [-15.5 ],
        [  7.25]])

#  Game Data  - Fall Sports

Find the game data for the four teams in the NFC North and NFC South.

The teams are:

* **NFC North:**  Green Bay Packers, Chicago Bears, Detroit Lions, Minnesota Vikings
* **NFC South:**  Atlanta Falcons, Carolina Panthers, New Orleans Saints, Tampa Bay Buccaneers


It may be helpful to do a google search to find the data.

Be sure to create your matrix on the handout, then enter the data for the system below.

# Game Data - Spring Sports

Find the game data for the seven teams in the Western Division for Men's Big Ten Basketball.

**The teams:**  Wisconsin, Minnesota, Iowa, Illinois, Nebraska, Purdue, and Northewstern.

I've provided **most** of the data, but you have to do a Google search to find the scores for the last two teams.

Be sure to create your matrix on the handout, then enter the data for the system below.


In [36]:
# create the matrix:
M = np.matrix([[1, 0, 0, 0, -1, 0, 0],
               [1, -1, 0, 0, 0, 0, 0],
               [1, 0, 0, 0, 0, 0, -1],
               [-1, 0, 0, 1, 0, 0, 0],             
               [1, 0, 0, 0, -1, 0, 0],          
               [-1, 0, 1, 0, 0, 0, 0],
               [1, 0, 0, 0, 0, 0, -1],             
               [-1, 0, 0, 1, 0, 0, 0],
               [-1, 0, 0, 0, 0, 1, 0],        
               [-1, 0, 1, 0, 0, 0, 0],
               [0, -1, 0, 1, 0, 0, 0],      
               [0, 1, -1, 0, 0, 0, 0],
               [0, -1, 1, 0, 0, 0, 0],              
               [0, -1, 0, 0, 0, 1, 0],
               [0, 1, 0, 0, -1, 0, 0],
               [0, 1, 0, 0, 0, -1, 0],
               [0, -1, 0, 1, 0, 0, 0],
               [0, -1, 0, 0, 0, 0, 1],
               [0, -1, 0, 0, 1, 0, 0],
               [0, 0, 1, 0, 0, -1, 0],
               [0, 0, 1, 0, 0, 0, -1],
               [0, 0, 1, 0, 0, 0, -1],
               [0, 0, -1, 1, 0, 0, 0],
               [0, 0, 1, 0, -1, 0, 0],
               [0, 0, 0, 1, 0, -1, 0],
               [0, 0, 0, 1, 0, 0, -1],
               [0, 0, 0, 1, -1, 0, 0],
               [0, 0, 0, 1, 0, 0, -1],
               [0, 0, 0, 1, -1, 0, 0],
               [0, 0, 0, 0, -1, 1, 0],
               [0, 0, 0, 0, -1, 0, 1],
               [0, 0, 0, 0, 0, 1, -1]])


# create the vector for the right hand side:
pts = np.matrix([[14], [12], [16], [-15], [13], [-15], [17], [-5], [-4], [-4], [-27], [7], [-15], [-19], [18], [3], [-31], [-8], [-4], [15], [15], [23], [-5], [38], [8], [25], [5], [7], [16], [-17], [-5], [5]])
 
#Make sure the system looks appropriate:
print(np.c_[M,pts])

[[  1   0   0   0  -1   0   0  14]
 [  1  -1   0   0   0   0   0  12]
 [  1   0   0   0   0   0  -1  16]
 [ -1   0   0   1   0   0   0 -15]
 [  1   0   0   0  -1   0   0  13]
 [ -1   0   1   0   0   0   0 -15]
 [  1   0   0   0   0   0  -1  17]
 [ -1   0   0   1   0   0   0  -5]
 [ -1   0   0   0   0   1   0  -4]
 [ -1   0   1   0   0   0   0  -4]
 [  0  -1   0   1   0   0   0 -27]
 [  0   1  -1   0   0   0   0   7]
 [  0  -1   1   0   0   0   0 -15]
 [  0  -1   0   0   0   1   0 -19]
 [  0   1   0   0  -1   0   0  18]
 [  0   1   0   0   0  -1   0   3]
 [  0  -1   0   1   0   0   0 -31]
 [  0  -1   0   0   0   0   1  -8]
 [  0  -1   0   0   1   0   0  -4]
 [  0   0   1   0   0  -1   0  15]
 [  0   0   1   0   0   0  -1  15]
 [  0   0   1   0   0   0  -1  23]
 [  0   0  -1   1   0   0   0  -5]
 [  0   0   1   0  -1   0   0  38]
 [  0   0   0   1   0  -1   0   8]
 [  0   0   0   1   0   0  -1  25]
 [  0   0   0   1  -1   0   0   5]
 [  0   0   0   1   0   0  -1   7]
 [  0   0   0   1  -

In [40]:
# Generate the transpose of M:
MT = M.transpose()

print("The Transpose of M is:")
print(MT)

# and find MT*M and MT*pts:
print("\n")
print("MT * M is: ")
MTM = MT * M
print(MTM)

print("\n")
print("MT * pts is:")
MTpts = MT * pts
print(MTpts)
print("\n\n",MTM)

The Transpose of M is:
[[ 1  1  1 -1  1 -1  1 -1 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  0  0  0  0  0 -1  1 -1 -1  1  1 -1 -1 -1  0  0  0  0  0
   0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  1  0 -1  1  0  0  0  0  0  0  1  1  1 -1  1
   0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  1  0  0  1  0  0  0  0  0  1  0  0  0  0  0  1  0
   1  1  1  1  1  0  0  0]
 [-1  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1  0  0  0  1  0  0  0  0 -1
   0  0 -1  0 -1 -1 -1  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  1  0 -1  0  0  0 -1  0  0  0  0
  -1  0  0  0  0  1  0  1]
 [ 0  0 -1  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  1  0  0 -1 -1  0  0
   0 -1  0 -1  0  0  1 -1]]


MT * M is: 
[[10 -1 -2 -2 -2 -1 -2]
 [-1 10 -2 -2 -2 -2 -1]
 [-2 -2  9 -1 -1 -1 -2]
 [-2 -2 -1 10 -2 -1 -2]
 [-2 -2 -1 -2  9 -1 -1]
 [-1 -2 -1 -1 -1  7 -1]
 [-2 -1 -2 -2 -1 -1  9]]


MT * pts is:
[[ 115]
 [ 120]
 [  55]
 [ -22]
 [ -86]
 [ -61]
 [-121]]


 [[10 -1 -2 

In [38]:
#  Check if MTM is singular
is_singular(MT*M)

True

In [41]:
# Careful - you might not need to do this!  
# Be sure to check whether MTM is singular on the previous line.

# If MTM is singular, then adjust one row of MTM and MT pts to be just 1's:

#MTM[3] is the entire 4th row of MTM, 
#so MTM[3] = 1 will put 1's in MTM's 4th row.

MTM[6] = 1
MTpts[6] = 1

print("MTM is now: ")
print(MTM)

print("and MTpts is now:")
print(MTpts)

print("Is our modified MTM singular?")
is_singular(MTM)

MTM is now: 
[[10 -1 -2 -2 -2 -1 -2]
 [-1 10 -2 -2 -2 -2 -1]
 [-2 -2  9 -1 -1 -1 -2]
 [-2 -2 -1 10 -2 -1 -2]
 [-2 -2 -1 -2  9 -1 -1]
 [-1 -2 -1 -1 -1  7 -1]
 [ 1  1  1  1  1  1  1]]
and MTpts is now:
[[115]
 [120]
 [ 55]
 [-22]
 [-86]
 [-61]
 [  1]]
Is our modified MTM singular?


False

In [42]:
#Find our solution using the inverse of MTM:

#First, find the inverse of MTM:
MTMI = np.linalg.inv(MTM)

print("The inverse of MTM is:")
print(MTMI)

#Now, use the inverse of MTM to solve for the rankings:

print("\n")
print("The solution are rankings:")
MTMI * MTpts

The inverse of MTM is:
[[ 8.32670976e-02 -8.67025223e-03 -7.21969055e-04 -7.28592624e-04
  -7.94828317e-04 -1.13108485e-02  1.42857143e-01]
 [ 7.06514060e-04  9.24826904e-02  7.70100325e-03  7.77165466e-03
   8.47816872e-03  9.53793981e-03  1.42857143e-01]
 [-7.77165466e-04 -1.73095945e-03  9.15288964e-02 -8.54882012e-03
  -9.32598559e-03 -1.04917338e-02  1.42857143e-01]
 [ 5.74042673e-05 -8.19114738e-04 -7.70762682e-03  8.39647803e-02
   6.88851208e-04 -9.64170906e-03  1.42857143e-01]
 [ 8.40310160e-03  8.29933234e-03 -7.28592624e-05  9.10078423e-03
   1.00837219e-01 -1.14146178e-03  1.42857143e-01]
 [ 8.83142575e-05  1.15603363e-02  9.62625406e-04  9.71456832e-04
   1.05977109e-03  1.26192242e-01  1.42857143e-01]
 [-9.17452664e-02 -1.01122033e-01 -9.16900699e-02 -9.25312632e-02
  -1.00943196e-01 -1.03144429e-01  1.42857143e-01]]


The solution are rankings:


matrix([[  9.41278084],
        [ 10.26367105],
        [  6.50996185],
        [ -1.69107673],
        [ -6.70146248],
        [ -6.21704112],
        [-10.5768334 ]])